#Análisis Fashion MNIST usando Pytorch Lightning

En este cuaderno se transcribe el código propuesto para una red neuronal que permita clasificar prendas de vestir empleando la base de datos Fashion Mnist. Para el análisis se va a usar Pytorch Lightning

In [1]:
!pip install pytorch_lightning
import pytorch_lightning as pl
from pytorch_lightning import LightningModule
from torch.autograd import Variable

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchmetrics import Accuracy
from datetime import datetime
from torch.nn.functional import softmax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Para la especificación del modelo se usa el modulo de lightning y se define la red neuronal que se va a ajustar.

In [2]:
class NeuralNetwork(pl.LightningModule):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


A continuación se define una para ejecutar el proceso de entrenamiento

In [3]:
class Trainer:
    def __init__(self, model, loss_fn, optimizer, metrics=None,
                 metric_names=None,
                 writer=None, path_to_save='',
                 learning_rate = 1e-3,
                 batch_size = 64,
                 epochs = 5, n_report= 1000):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.metrics_train = metrics
        self.metrics_valid = self.metrics_train.copy()
        self.metric_names = metric_names

        self.writer = writer
        
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.n_report = n_report
        
        self.path_to_save = path_to_save

        self.best_model = None
        
        self.training_loader = None

        self.validation_loader = None
     
    def set_model(self, model):
        self.model = model
    
    def set_loss(self, loss):
        self.loss = loss
        
    def set_optimizer(self, optimizer):
        self.optimizer = optimizer
        
    def set_writer(self, writer):
        self.writer =  writer

    def get_model(self):
        return self.model
    
    
    def set_hiperparameters(self,
                 learning_rate = 1e-3,
                 batch_size = 64,
                 epochs = 5):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs        
    
    def _train_one_epoch_(self, epoch_index):
        running_loss = 0.
        last_loss = 0.

        for i, data in enumerate(self.training_loader):

            inputs, labels = data

            self.optimizer.zero_grad()

            outputs = self.model(inputs)
            predicts = softmax(outputs, dim=-1)

            loss = self.loss_fn(outputs, labels)
            loss.backward()

            self.optimizer.step()

            running_loss += loss.item()

            running_metrics = self._metric_step_(predicts, labels, metric_compute=False, 
                                          validation=False)
            
            if i % self.n_report == (self.n_report-1):

                last_loss = running_loss / self.n_report
                running_loss = 0.

                last_metrics = self._metric_step_(None, None, metric_compute=True, 
                                          validation=False)
                
                print('Pérdida en el lote {} : {}'.format(i + 1, last_loss))
                
                print_m = ''
                for j in range(len(metrics)):
                    print_m += self.metric_names[j] + ': ' + str(last_metrics[j]) + ' '
                print('Métricas en el lote {} : {}'.format(i + 1, print_m))
                
                if self.writer is not None:
                    tb_x = epoch_index * len(training_loader) + i + 1

                    self.writer.add_scalar('Pérdida/Entrenamiento', last_loss, tb_x)

                    for i in range(len(last_metric)):
                        self.writer.add_scalar(self.metric.names[i] + '/Entrenamiento', 
                                               last_metrics[i], tb_x)
                        
                
                
    def _validation_step_(self, validation=True):

        if validation:
            data_loader = self.validation_loader
        else:
            data_loader = self.training_loader
        
        running_vloss = 0.0   
        for i, vdata in enumerate(data_loader):
            vinputs, vlabels = vdata
            voutputs = self.model(vinputs)
            vpredicts = softmax(voutputs,dim=-1)
            vloss = self.loss_fn(voutputs, vlabels)
            
            running_vloss += vloss
            _  = self._metric_step_(vpredicts, vlabels, 
                            metric_compute=False, validation= validation)

        avg_vloss = running_vloss / (i + 1)
        v_metrics =   self._metric_step_(None, None, metric_compute=True, 
                                          validation= validation)
        
        return avg_vloss, v_metrics
        
    def _metric_step_(self, predicts, labels, metric_compute=False, validation=False):

        if validation:
            metrics = self.metrics_valid
        else:
            metrics = self.metrics_train
        
        if predicts is not None and labels is not None:
            for i, metric in enumerate(metrics):
                metrics[i].update(predicts, labels)
        
        if metric_compute:
            values = [metric.compute().item() for metric in metrics]
            for metric in metrics:
                metric.reset() 
        else:
            values = [metric(predicts, labels).item() for metric in metrics]
    
        return values
      
    
    def _train_loop_(self):

        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        epoch_number = 0


        best_vloss = 1_000_000.
        
         

        for epoch in range(self.epochs):
            print('época {}:'.format(epoch_number + 1))
            

            model.train(True)

            self._train_one_epoch_(epoch_number)


            self.model.train(False)
            
            e_loss, e_metrics = self._validation_step_(validation=False)
            v_loss, v_metrics = self._validation_step_(validation=True)
            
            print('Pérdida entrenamiento: {}, validación {}'.format(e_loss, v_loss))
            
            print_m_e = ''
            for i in range(len(e_metrics)):
                print_m_e += self.metric_names[i] + ': ' + str(e_metrics[i]) + ' '
            print_m_v = ''
            for i in range(len(v_metrics)):
                print_m_v += self.metric_names[i] + ': ' + str(v_metrics[i]) + ' '     
            print('Métricas en entrenamiento : {}, validación {} '.format(print_m_e, print_m_v))


            if self.writer is not None:
                self.writer.add_scalars('Pérdida entrenamiento v.s. Pérdida validación',
                                { 'Entrenamiento' : e_loss, 'Validación' : e_vloss },
                                epoch_number + 1)
                
                for i in range(len(e_metrics)):  
                    self.writer.add_scalars(self.metric.names[i] + 'entrenamiento v.s. validación',
                                { 'Entrenamiento' : e_metrics[i], 'Validación' : v_metrics[i] },
                                epoch_number + 1)
                  
                self.writer.flush()

            if  v_loss < best_vloss:
                best_vloss = v_loss
                model_path = self.path_to_save + 'model_{}_{}'.format(timestamp, epoch_number)
                torch.save(model.state_dict(), model_path)
                self.path_best_model = model_path

            epoch_number += 1 

       
                    
            
    def fit(self, train_data, val_data, epochs=None, writer=None, best_loss=True):
        if writer is not None:
            self.writer = writer
        if epochs is not None:
            self.epochs = epochs

        self.training_loader = train_data

        self.validation_loader = val_data
        

        self._train_loop_()

        if best_loss:
            self.model.load_state_dict(torch.load(self.path_best_model))

Adicional se define una clase para generar el conjunto de entrenamiento y validación

In [4]:
class Data():
    def __init__(self, dataset=None, batch_size=64, shuffle=True):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self._data = DataLoader(dataset, batch_size=self.batch_size, shuffle=self.shuffle)
    
    # getter
    def get_data(self):
        return self._data
    
    # setter
    def set_data(self, dataset):
        self._dataset = dataset
        self._data = DataLoader(self._dataset, batch_size=self.batch_size, shuffle=self.shuffle)
    
    # crea la propiedad data
    data = property(get_data, set_data)
    
    def __len__(self):
        return len(self._data)
    

Se generan los conjuntos de entramiento y prueba

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

In [6]:
train = Data(train_data, batch_size=32)
validation = Data(test_data, batch_size=32, shuffle=False)

Ahora se definen los optimizadores y las funciones de pérdida. Debido a que la red que se está entrenando es realtivamente compleja, se guarda el checkpoint para usarlo con otros datos.

In [7]:
model = NeuralNetwork()

# Optimizador
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Función de pérdida
loss_fn = torch.nn.CrossEntropyLoss()

# initializa métrica
# se espera una lista de métricas
metrics = [Accuracy()] # accuracy
# pasar nombre en español de la métrica
# TODO hacer esto con una clase traductora
metric_names = ['Exactitud']
# path para almacenar los pesos de los mejores modelos
from google.colab import drive
drive.mount('/content/gdrive')

path_to_save = '/content/gdrive/MyDrive/Colab Notebooks'

# Trainer
trainer = Trainer(model=model, loss_fn=loss_fn,  
                  optimizer=optimizer, metrics=metrics, 
                  metric_names = metric_names,
                  n_report=375, path_to_save= path_to_save )

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Se ajusta el modelo de entramiento a los datos de validación para determinar su precisión

In [8]:
trainer.fit(train.data, validation.data)

época 1:
Pérdida en el lote 375 : 1.4949666884740194
Métricas en el lote 375 : Exactitud: 0.4857499897480011 
Pérdida en el lote 750 : 1.1020420747598012
Métricas en el lote 750 : Exactitud: 0.6317499876022339 
Pérdida en el lote 1125 : 0.808643186767896
Métricas en el lote 1125 : Exactitud: 0.746916651725769 
Pérdida en el lote 1500 : 0.6376968868970871
Métricas en el lote 1500 : Exactitud: 0.8212500214576721 
Pérdida en el lote 1875 : 0.6097861173550287
Métricas en el lote 1875 : Exactitud: 0.8303333520889282 
Pérdida entrenamiento: 0.5756624341011047, validación 0.6217970252037048
Métricas en entrenamiento : Exactitud: 0.8401833176612854 , validación Exactitud: 0.822700023651123  
época 2:
Pérdida en el lote 375 : 0.6160221873521805
Métricas en el lote 375 : Exactitud: 0.8335833549499512 
Pérdida en el lote 750 : 0.5837854881683986
Métricas en el lote 750 : Exactitud: 0.8454166650772095 
Pérdida en el lote 1125 : 0.5674816248615583
Métricas en el lote 1125 : Exactitud: 0.85241669416

El modelo ajustado cuenta con una exactitud del 90.34% para el conjunto de entrenamiento y 87.48% en el de validación lo cual es bueno teniendo en cuenta que no se usa una red convolucional.